In [15]:
# # Custom Estimator and export
# https://github.com/tensorflow/hub/issues/132
#nnlm_module_path,uni_sent_module_path
#uni_sent_module_path='/media/sf_codebase/modules/nnlm-en-dim128'

import tensorflow as tf
from tensorflow.keras.models import Model
import tensorflow_hub as hub
print ("Tensorflow, hub versions",tf.__version__,hub.__version__)

_TEXT_FEATURE_NAME = "text"
_EXPORT_MODULE_NAME = "unifinetune"

uni_sent_module_path="https://tfhub.dev/google/nnlm-en-dim128-with-normalization/1"

def _input_fn():
    """An input fn."""
    features = {
        _TEXT_FEATURE_NAME: tf.constant([
          "hadoop hive", "asp net","js javascript nodejs"]),
    }
    labels = tf.constant([0,1,2])
    return features, labels


def _serving_input_fn():
    """A serving input fn."""
    text_features = tf.placeholder(dtype=tf.string, shape=[None])
    return tf.estimator.export.ServingInputReceiver(
        features={_TEXT_FEATURE_NAME: text_features},
        receiver_tensors=text_features)

def _get_model_fn(register_module=False):
    def _model_fn(features, labels, mode):
        """A model_fn that uses a mock TF-Hub module."""
#         del labels
        
        hub_module = hub.Module(uni_sent_module_path,trainable=True)
        hub.register_module_for_export(hub_module, "my_module")
        
        embedding = hub_module(features["text"])
        print (embedding.shape)
        first_layer = tf.layers.dense(inputs=embedding, units=100, activation=tf.nn.relu)
        second_layer = tf.layers.dense(inputs=first_layer, units=100, activation=tf.nn.relu)
        logits = tf.layers.dense(inputs=second_layer, units=3, activation=tf.nn.softmax)
        predictions = tf.argmax(logits, 1)
#         labels = tf.cast(labels, tf.int64)
#         print (labels.shape,logits.shape,predictions.shape)
        labels=labels
        print (labels)
        # Create an optimizer that will take care of the Gradient Descent
        optimizer = tf.train.AdamOptimizer(0.01)

        if mode == tf.estimator.ModeKeys.PREDICT:
            print ("I am in mode:",mode)
            return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)
        
        # Create the training operation
        loss = tf.losses.sparse_softmax_cross_entropy(labels,logits)
        train_op = optimizer.minimize(loss,tf.train.get_global_step())
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            print ("I am in mode:",mode)
            return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions,loss=loss,train_op=train_op)
    return _model_fn

import os
os.makedirs("modeldir", exist_ok=True)
os.makedirs("export", exist_ok=True)
model_dir = os.path.join(os.getcwd(),'modeldir')
export_base_dir = os.path.join(os.getcwd(), "export")
estimator = tf.estimator.Estimator(_get_model_fn(register_module=True),
                                       model_dir=model_dir)
estimator.train(input_fn=_input_fn, steps=10)
exporter = hub.LatestModuleExporter("exporter_name", _serving_input_fn)
export_dir = exporter.export(estimator=estimator,export_path=export_base_dir,checkpoint_path=estimator.latest_checkpoint(),eval_result=None,is_the_final_export=None)
exported_module_path=os.path.join(export_dir.decode("utf-8"),"my_module")

Tensorflow, hub versions 1.12.0 0.2.0
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_evaluation_master': '', '_experimental_distribute': None, '_protocol': None, '_device_fn': None, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f95587249b0>, '_task_id': 0, '_tf_random_seed': None, '_eval_distribute': None, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_save_summary_steps': 100, '_master': '', '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_train_distribute': None, '_service': None, '_model_dir': '/media/sf_codebase/experiment/modeldir', '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_is_chief': True, '_global_id_in_cluster': 0, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there

In [19]:
print ("With finetuning we expect the related technologies should come near to each other,reflecting local corpus")

# %%time
# #https://github.com/tensorflow/hub/blob/master/docs/common_issues.md#running-inference-on-a-pre-initialized-module
import pandas as pd
import numpy as np

import numpy as np
import pandas as pd
test=pd.read_excel('test.xlsx')
test=test['Curated Risks'].values.tolist()
test=["hadoop","hive", "asp", "net","js","javascript","nodejs"]
exported_module_path=exported_module_path
print ('module path',exported_module_path)
import tensorflow as tf
import tensorflow_hub as hub

# Create graph and finalize (finalizing optional but recommended).
g = tf.Graph()
with g.as_default():
  # We will be feeding 1D tensors of text into the graph.
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  #nnlm-en-dim128,universal_sentence_encoder_large_3
  embed = hub.Module(exported_module_path)
  embedded_text = embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Create session and initialize.
session = tf.Session(graph=g)
session.run(init_op)
def get_embeddings(embedded_text,session,text):
    result = session.run(embedded_text, feed_dict={text_input: text})
    import pandas as pd
    text_embeddings=pd.DataFrame(result)
    return(text_embeddings)
doc_embed=get_embeddings(embedded_text,session,test)
# doc_embed.to_pickle('doc_embed_us3.pkl')

def get_similiarity(target_text_embed,text_to_compare_embed):
    from sklearn.metrics.pairwise import cosine_similarity
    similiarity=cosine_similarity(target_text_embed,text_to_compare_embed)
    similiarity=pd.DataFrame(similiarity)
    return similiarity
sim=get_similiarity(doc_embed,doc_embed)
np.fill_diagonal(sim.values, 0)
# idx=sim.idxmax(axis=1).values[0]
# sim[idx]

sim.index=test
sim.columns=test
y=sim.idxmax(axis=1).values.tolist()
x=test
z=sim.max(axis=1).values.tolist()
pd.set_option('display.max_colwidth', -1)
data_finetuned=pd.DataFrame([x,y,z]).T
# data.to_csv('u3_u4.csv')
data_finetuned

With finetuning we expect the related technologies should come near to each other,reflecting local corpus
module path /media/sf_codebase/experiment/export/1575045661/my_module
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


,0,1,2
0,hadoop,hive,0.449132
1,hive,hadoop,0.449132
2,asp,net,0.481343
3,net,asp,0.481343
4,js,javascript,0.673985
5,javascript,js,0.673985
6,nodejs,js,0.517689


In [20]:
print ("Without finetuning we expect the related technologies should be far,reflecting global or random vector assignment for Out of Vocabulary using Hash Tables")


# %%time
# #https://github.com/tensorflow/hub/blob/master/docs/common_issues.md#running-inference-on-a-pre-initialized-module
import pandas as pd
import numpy as np

import numpy as np
import pandas as pd
test=pd.read_excel('test.xlsx')
test=test['Curated Risks'].values.tolist()
test=["hadoop","hive", "asp", "net","js","javascript","nodejs"]

uni_sent_module_path="https://tfhub.dev/google/nnlm-en-dim128-with-normalization/1"
print ('uni_sent_module_path',uni_sent_module_path)
import tensorflow as tf
import tensorflow_hub as hub

# Create graph and finalize (finalizing optional but recommended).
g = tf.Graph()
with g.as_default():
  # We will be feeding 1D tensors of text into the graph.
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  #nnlm-en-dim128,universal_sentence_encoder_large_3
  embed = hub.Module(uni_sent_module_path,trainable=True)
  embedded_text = embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Create session and initialize.
session = tf.Session(graph=g)
session.run(init_op)
def get_embeddings(embedded_text,session,text):
    result = session.run(embedded_text, feed_dict={text_input: text})
    import pandas as pd
    text_embeddings=pd.DataFrame(result)
    return(text_embeddings)
doc_embed=get_embeddings(embedded_text,session,test)
# doc_embed.to_pickle('doc_embed_us3.pkl')

def get_similiarity(target_text_embed,text_to_compare_embed):
    from sklearn.metrics.pairwise import cosine_similarity
    similiarity=cosine_similarity(target_text_embed,text_to_compare_embed)
    similiarity=pd.DataFrame(similiarity)
    return similiarity
sim=get_similiarity(doc_embed,doc_embed)
np.fill_diagonal(sim.values, 0)
# idx=sim.idxmax(axis=1).values[0]
# sim[idx]

sim.index=test
sim.columns=test
y=sim.idxmax(axis=1).values.tolist()
x=test
z=sim.max(axis=1).values.tolist()
pd.set_option('display.max_colwidth', -1)
data_global=pd.DataFrame([x,y,z]).T
# data.to_csv('u3_u4.csv')
data_global

Without finetuning we expect the related technologies should be far,reflecting global or random vector assignment for Out of Vocabulary using Hash Tables
uni_sent_module_path https://tfhub.dev/google/nnlm-en-dim128-with-normalization/1
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


,0,1,2
0,hadoop,nodejs,0.0677032
1,hive,net,0.154901
2,asp,javascript,0.266492
3,net,hive,0.154901
4,js,javascript,0.403418
5,javascript,js,0.403418
6,nodejs,js,0.141451


In [21]:
data_finetuned.columns=['word','Fine tuned most similiar','Finetuned_similiarity']
data_global.columns=['word','Global most similiar','Global similiarity']
data_global.merge(data_finetuned,how='left',on='word')

,word,Global most similiar,Global similiarity,Fine tuned most similiar,Finetuned_similiarity
0,hadoop,nodejs,0.0677032,hive,0.449132
1,hive,net,0.154901,hadoop,0.449132
2,asp,javascript,0.266492,net,0.481343
3,net,hive,0.154901,asp,0.481343
4,js,javascript,0.403418,javascript,0.673985
5,javascript,js,0.403418,js,0.673985
6,nodejs,js,0.141451,js,0.517689
